<h1><center>

![](https://i.imgur.com/jrRGMfx.png)

AI PRACTITIONER COURSE 

PRACTICE LAB - SESSION 16 - Interactive decision making: Contextual & Multi-armed Bandits

**&copy; 2020 VTCA-COTAI. Internal Use Only.**

</center></h1>


## <a id='mab'>Phân tích bài toán Multi-Armed Bandit với Showing advertisement.</a>
Ví dụ, tưởng tượng rằng ta đang thực hiện chiến dịch marketing cho trang web với hai lựa chọn chạy quảng cáo. Giả sử mục tiêu là hiển thị quảng cáo có tỉ lệ nhấp chuột CTR (click through rate) cao nhất để thu hút lượng truy cập cao nhất có thể. Nhưng ta không có bất kỳ thông tin trước đó để biết là nên hiển thị mấy cái đó như thế nào. Cách điển hình là sẽ thử nghiệm chiếu cả hai như nhau, tính toán CTR của từng cái, sau đó, ở một vài thời điểm sẽ chuyển sang quảng cáo có CTR cao hơn. Ta sẽ chiếu cả hai quảng cáo trong bao lâu trước khi tập trung chiếu ổn định một quảng cáo trội hơn (để thu lợi nhiều hơn)? Trong những trường hợp này, có vẻ như đoán mò có lẽ là cách đánh cược tốt nhất. Thực tế thì đó là cách thường dùng. 

Phương pháp ta dùng để giải bài toán này gọi là `Tướng cướp nhiều tay` (multi armed bandit - MAB). (Và thường người ta cũng gọi bài toán trên là bài toán MAB)

**Các thông số cần nhớ**
- **CTR** là tỷ lệ giữa số lần click chuột với số lần hiển thị của một quảng cáo. Ví dụ, nếu một quảng cáo được chiếu 100 lần và được click xem 10 lần thì CTR = 10/100 = 0.1.

- **Regret** là sự khác nhau giữa CTR khả dĩ cao nhất và CTR tính được. Ví dụ, nếu quảng cáo A có CTR đã biết là 0.1 và quảng cáo B có CTR đã biết là 0.3, mỗi khi ta chiếu quảng cáo A, ta có regret bằng 0.3 - 0.1 = 0.2. Đây có vẻ là một sự khác nhau nho nhỏ cho đến khi ta xét đến một quảng cáo có thể được hiển thị 1 triệu lần chỉ trong vài giờ.

Tiếp theo, ta sẽ thực thi để tìm ra thuật toán nào sẽ tốt nhất trong việc tối thiểu hóa regret.
Bốn cách thực thi mà ta sẽ dùng gồm:

1. Chọn Ngẫu Nhiên (Random Selection)
2. Epsilon Greedy
4. Cận trên của khoảng tin cậy (Upper Confidence Bound - UCB1)
3. Lấy mẫu Thompson

Sau đây ta sẽ mô phỏng lại 4 cách trên.

Giả sử rằng ta biết trước CTR. Như vậy, ta có thể mô phỏng một cú click (hoặc không) của một quảng cáo cho trước. Ví dụ, nếu ta chiếu quảng cáo A, với CTR đã biết là 28%, ta có thể giả sử rằng quảng cáo sẽ được click chuột trong 28% thời gian hiển thị và đưa nó vào mô phỏng bên dưới.


##Import thư viện

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import beta, bernoulli
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import subplots
init_notebook_mode(connected=True)
import random
import math
import plotly
plotly.io.renderers.default = 'colab'

# Các hàm hỗ trợ

Ở đây, ta sẽ giả sử mình biết được tỉ lệ click thật sự của mỗi quảng cáo để thực hiện giả lập.

Thực tế, bạn sẽ cần implement + deploy 1 hệ thống hoàn chỉnh có khả năng lưu lại lịch sử click quảng cáo của khách hàng

In [ ]:
# Để mô phỏng ta giả sử biết giá trị CTR
ACTUAL_CTR = [.45, .65]
print('Actual CTR for Ad #0 is:', ACTUAL_CTR[0])
print('Actual CTR for Ad #1 is:', ACTUAL_CTR[1])

Actual CTR for Ad #0 is: 0.45
Actual CTR for Ad #1 is: 0.65


Hàm ``simulate_customer`` để giả lập việc khách hàng có click vào quảng cáo hay không ?
- Nhận vào ``prob`` xác suất click
- Return ``True`` hoặc ``False`` dại điện cho việc khách hàng có click hay không


In [ ]:
# prob: the probability that customer will click on the ad
def simulate_customer(prob):

  # TODOs: 
  # Tìm cách dùng hàm bernoulli.rvs() của thư viện scipy
  # Return ra kết quả

    r = bernoulli.rvs(prob)

    return r
# simulate_customer(0.5)

Class ``Stat`` để ghi lại các con số thống kê cần thiết

In [ ]:
did_list = [1,2,3]
did_list[0] +=1
did_list

[2, 2, 3]

In [ ]:
class Stat:
  def __init__(self):
    # danh sách ghi lại quảng cáo nào được chiếu
    self.ad_index = []
    # array gồm 2 phần tử, đại diện cho tổng số lần mỗi quảng cáo được chiếu
    self.impressions = [0,0]
    # array gồm 2 phần tử, đại diện cho mỗi quảng cáo được click bao nhiêu lần
    self.clicks = [0,0]
    # tổng regret
    self.regret = 0
    # tổng reward 
    self.total_reward = 0
    ## danh sách ghi lại các giá trị regret trong mỗi lượt chiếu
    self.regret_list = []
    ## dict gồm 2 phần tử, dùng để ghi lại ctr tính được cho mỗi quảng cáo
    self.ctr = {0: [], 1: []}

  def record(self, ad_index, did_click, ctrs):
    # TODO: Ghi lại lịch sử chuỗi quảng được chọn chiếu
    # TODO: Thống kê lại số lượt chiếu của mỗi quảng cáo
    # TODO: Thống kê lại số lần khách hàng click vào mỗi quảng cáo
    # TODO: Thống kê lại tổng regret và reward

    self.ad_index.append(ad_index)

    self.impressions[ad_index] += 1

    if did_click  : 
        self.clicks[ad_index] +=1
        self.total_reward +=1

    regret = max(ACTUAL_CTR)- ACTUAL_CTR[ad_index]
    self.regret += regret
    self.regret_list.append(self.regret) 


    if ctrs and len(ctrs) > 0:
        self.ctr[0].append(ctrs[0])
        self.ctr[1].append(ctrs[1])
    else:
        ## tính giá trị CTR và thêm nó vào danh sách
        if self.impressions[0] == 0:
            ctr_0 = 0
        else:
            ctr_0 = self.clicks[0]/self.impressions[0]
        if self.impressions[1] == 0:
            ctr_1 = 0
        else:
            ctr_1 = self.clicks[1]/self.impressions[1]    
        self.ctr[0].append(ctr_0)
        self.ctr[1].append(ctr_1)
    

  def export_stat_dict(self):
      return {
          'reward': self.total_reward, 
          'regret_list': self.regret_list, 
          'ads_count': pd.Series(self.ad_index).value_counts(normalize=True)
      }


Hàm vẽ chart để biểu diễn các thông số trên

In [ ]:
def algorithm_performance(stat):
    """
    Hàm biểu diễn việc thực hiện mỗi thuật toán ta sẽ dùng trong phần hướng dẫn này.
    """
    count_series = pd.Series(stat.ad_index).value_counts(normalize=True)
    print('Ad #0 has been shown', count_series[0]*100, '% of the time.')
    print('Ad #1 has been shown', count_series[1]*100, '% of the time.')
    
    print('Total Reward (Number of Clicks):', stat.total_reward) ## print total Reward
    
    x = np.arange (0, 1000, 1)

    # tính và vẽ đồ thị CTR đã tính được cho quảng cáo #0
    data1 = go.Scatter(x=x,
                       y=stat.ctr[0],
                       name='Calculated CTR #0',
                       line=dict(color=('rgba(10, 108, 94, .7)'),
                                 width=2))

    ## vẽ đường thẳng biểu diễn giá trị CTR thực tế của quáng cáo #0
    data2 = go.Scatter(x=[0, 1000],
                       y=[ACTUAL_CTR[0]] * 2,
                       name='Actual CTR #0 value',
                       line = dict(color = ('rgb(205, 12, 24)'),
                                   width = 1,
                                   dash = 'dash'))

    ## tính và vẽ đồ thị CTR đã tính được cho quảng cáo #1
    data3 = go.Scatter(x=x,
                       y=stat.ctr[1],
                       name='Calculated CTR #1',
                       line=dict(color=('rgba(187, 121, 24, .7)'),
                                 width=2))

    ## vẽ đường thẳng biểu diễn giá trị CTR thực tế của quáng cáo #1
    data4 = go.Scatter(x=[0, 1000],
                       y=[ACTUAL_CTR[1]] * 2,
                       name='Actual CTR #1 value',
                       line = dict(color = ('rgb(205, 12, 24)'),
                                   width = 1,
                                   dash = 'dash'))

    ## biểu diễn giá trị Regret: là một hàm có tham số là số lần thử nghiệm
    data5 = go.Scatter(x=x,
                       y=stat.regret_list,
                       name='Regret')

    layout = go.Layout(title='Simulated CTR Values and Algorithm Regret',
                       xaxis={'title': 'Trial Number'},
                       yaxis1={'title': 'CTR value'},
                       yaxis2={'title': 'Regret Value'}
                       )
    fig = subplots.make_subplots(rows=2, cols=1, print_grid=False, shared_yaxes=True, shared_xaxes=True)

    fig.append_trace(data1, 1, 1)
    fig.append_trace(data2, 1, 1)
    fig.append_trace(data3, 1, 1)
    fig.append_trace(data4, 1, 1)
    fig.append_trace(data5, 2, 1)

    fig['layout'].update(layout)
    iplot(fig, show_link=False)

# <a id='random'>Chọn Ngẫu Nhiên (Random Selection)</a> 

Thuật toán Chọn ngẫu nhiên không làm chuyện khai phá, nó chỉ chọn ngẫu nhiên quảng cáo để hiển thị.

Bạn có thể hình dung như việc tung đồng xu vậy - nếu mặt ngửa ta chiếu quảng cáo #0, nếu mặt sấp ta chiếu quảng cáo #1. Vì vậy nếu ta có 2 quảng cáo, mỗi quảng cáo sẽ xuất hiện ~50% (=100%/2) số lần trình chiếu. Đến đây có thể bạn đã đoán được hạn chế của giải thuật này là gì rồi, nhưng hãy thử chạy mô phỏng xem thế nào.

## Prepare

In [ ]:
def do_random_selection():
  # TODO
  # 1. Random ra số 0 hoặc 1, đại diện cho việc show ra quảng cáo nào
  # 2. Return số đó ra
    rand = np.random.choice([0,1])
    return rand
do_random_selection()


0

Giả lập thuật toán ``Random Selection`` 1000 lần. Với mỗi lần chạy, ta gọi hàm ``record`` của class Stat để lưu lại kết quả

In [ ]:
np.random.seed(42)

stat_random = Stat()

# TODOs
for i in range(1000) :
    a = do_random_selection()
    b = simulate_customer(ACTUAL_CTR[a])
    stat_random.record(a,b,ctrs = None)

## Benchmark

In [ ]:
algorithm_performance(stat_random)

Ad #0 has been shown 50.6 % of the time.
Ad #1 has been shown 49.4 % of the time.
Total Reward (Number of Clicks): 555


Cả hai quảng cáo đều được chiếu với số lần bằng nhau và qua càng nhiều lượt thử nghiệm thì giá trị CTR tính toán được sẽ càng gần với giá trị mà ta đã biết trước. Tuy nhiên, regret vẫn liên tục tăng do thuật toán không học được gì và không cập nhật gì so với thông tin thu được. Regret đang tăng lên này chính xác là những gì ta hy vọng sẽ giảm thiểu được bằng các phương pháp "thông minh hơn".

# <a id='epsilon'>Epsilon Greedy</a> 

Cách tiếp cận tiếp theo là thuật toán Epsilon-Greedy. Logic như sau:

1. Chạy thử nghiệm mỗi quảng cáo ``n`` lần (**Khai phá**).

2. Trong giai đoạn khai phá ban đầu này, chọn ra quảng cáo có số điểm cao nhất.
3. Thiết lập giá trị Epsilon,  **$\epsilon$**.
4. Với mỗi lần chạy tiếp theo, ta sẽ random 1 con số bất kì trong khoảng $(0, 1)$, nếu con số này $<= \epsilon$  , thì ta thực hiện **khai phá**, ngược lại ta thực hiện **khai thác**



## Prepare

In [ ]:
np.random.seed(42)

stat_epsilon_greedy = Stat()

# TODOs
# 1. chạy thử mỗi quảng cáo 10 lần, sau đó chọn ra quảng cáo được click nhiều nhất
# 2. biến win_ad_index là index của quảng cáo có ctr cao nhất
for i in range(2):
    for j in range(10):
        b = simulate_customer(ACTUAL_CTR[i])
        stat_epsilon_greedy.record(i,b,ctrs = None)
click_max =np.argmax(stat_epsilon_greedy.clicks)

ctr_0 = stat_epsilon_greedy.ctr[0]
ctr_1 = stat_epsilon_greedy.ctr[1]

# if ctr_0 > ctr_1 :
#     win_ad_index = 0
# else:
#     win_ad_index = 1
win_ad_index = np.argmax(stat_epsilon_greedy.ctr)
lose_index = np.argmin(stat_epsilon_greedy.ctr)


print('After 20  initial trials Ad #', win_ad_index, 'got the highest CTR =', round(np.max([ctr_0, ctr_1]), 2), 
      '(Real CTR value is', ACTUAL_CTR[win_ad_index], ').')

After 20  initial trials Ad # 0 got the highest CTR = 1.0 (Real CTR value is 0.45 ).


In [ ]:
epsilona = np.random.uniform(0,1)
print(epsilona)

0.07513778173580876


In [ ]:
np.random.seed(42)
e = 0.15
# Với mỗi lần chạy tiếp theo, ta sẽ random 1 con số bất kì trong khoảng  (0,1) , nếu con số này  <=ϵ  , thì ta thực hiện khai phá, ngược lại ta thực hiện khai thác
def do_epsilon_greedy(win_index, lose_index, epsilon):
    # TODOs: 
    # Lấy mẫu để quyết định xem lượt này là sẽ là khai phá (explore) hay khai thác (exploit)
    epsilona = np.random.uniform(0,1)
    if epsilona < epsilon :
        return lose_index
    return win_index

# TODOs
# Thực hiện giả lập 980 lần chạy
for i in range(980) :
    a = do_epsilon_greedy(win_ad_index,1-win_ad_index,e)
    b = simulate_customer(ACTUAL_CTR[a])
    stat_random.record(a,b,ctrs = None)


## Benchmark

In [ ]:
algorithm_performance(stat_epsilon_greedy)

Ad #0 has been shown 50.0 % of the time.
Ad #1 has been shown 50.0 % of the time.
Total Reward (Number of Clicks): 14


Lưu ý, nhìn vào đồ thị regret, ta nhận ra regret đã giảm thế nào! Thuật toán Epsilon-Greedy có vẻ thực thi tốt hơn Chọn ngẫu nhiên. Theo **Luật số lớn (Law of large numbers - LLN)** thì ta càng làm nhiều lượt thử nghiệm ban đầu, ta càng có thể chọn được quảng cáo trội hơn chính xác. Nhưng trong Marketing không phải lúc nào ta cũng muốn dựa vào cơ hội và hy vọng chạm được 'số lần thử nghiệm lớn' kia. 

> - Theo lý thuyết thống kê, **Luật số lớn** là một định lý mô tả kết quả của việc thực thi cùng một thử nghiệm với số lần lớn. Và theo đó, trung bình của các kết quả thu được từ số lượng lớn các thử nghiệm  xu hướng gần với giá trị kỳ vọng, và sẽ càng gần hơn khi ta thực thi nhiều lượt thử nghiệm hơn nữa. 

- Ưu điểm: Ta có thể kiểm soát tần suất hiển thị quảng cáo trội hơn sau các lượt thử nghiệm ban đầu bằng cách chọn các giá trị $\epsilon$ khác nhau.

- Khuyết điểm: Ta không thể biết được ϵ tối ưu nhất cho bài toán của bạn. Nếu ϵ của bạn quá lớn, nghĩa là bạn chọn số lần thử dành cho mục đích khai thác quá nhiều, nhưng nếu số ϵ của bạn quá nhỏ, bạn sẽ có khả năng đang khai phá không hiệu quả.

# <a id='ucb'>Cận trên của khoảng tin cậy (Upper Confidence Bound - UCB)</a> 


UCB quan tâm nhiều hơn về tính không chắc chắn (độ lệch lớn) của mỗi biến thể. Càng không chắc chắn về một quảng cáo, thì càng phải ưu tiên tập trung khai phá.

Thuật toán chọn ra biến thể với giá trị cận trên của khoảng tin cậy cao nhất (UCB) - cũng có nghĩa là dự đoán biến thể cho reward cao nhất. Được định nghĩa như sau:

$UCB = \bar x_i + 2 \sqrt{\frac{\log{t}}{n}}$ ,

trong đó:
- $\bar x_i$ - tỷ lệ CTR tại bước thứ $i$,

- $t$ - tổng số lượt chiếu của tất cả quảng cáo,

- $n$ - tổng số lượt chiếu của quảng cáo được chọn


Logic khá là dễ hiểu:

1. Tính UCB của tất cả biến thể.
2. Chọn biến thể với UCB cao nhất.
3. Thực hiện bước 1.

## Prepare

In [ ]:
np.random.seed(42)

def ucb(stat, cur_iter):
  ad_index = 0
  max_upper_bound = 0
  for k in [0, 1]:
    if stat.impressions[k] > 0:
      CTR = stat.ctr[k][-1]
      delta = 2 * math.sqrt(math.log(cur_iter + 1) / stat.impressions[k])
      upper_bound = CTR + delta
    else:
      upper_bound = 1e400
    if upper_bound > max_upper_bound:
      max_upper_bound = upper_bound
      ad_index = k
  return ad_index

stat_ucb = Stat()

for i in range(1000):
    ad_index = ucb(stat_ucb, i)
    did_click = simulate_customer(ACTUAL_CTR[ad_index])
    stat_ucb.record(ad_index, did_click, None)

## Benchmark

In [ ]:
algorithm_performance(stat_ucb)

Ta có thể thấy rằng regret tăng khi thuật toán cố gắng giảm độ không chắc chắc của CTR của quảng cáo #0 bằng cách chọn chiếu nó nhiều hơn.

Nó có thể hữu dụng khi ta muốn mô hình chọn được quảng cáo tốt nhất thường xuyên hơn, nhưng vẫn quan tâm đến việc giảm sự độ không chắc chắn của cả hai quảng cáo.

# <a id='ts'>Lấy mẫu Thompson</a> 


Phần khai phá của Lấy mẫu Thompson tinh vi hơn thuật toán $\epsilon$-Greedy. Ta xài **phân phối Beta (Beta distribution)**\* ở đây, tuy nhiên Lấy mẫu Thompson có thể được khái quát hóa để lấy mẫu từ bất kỳ phân phối nào trên các tham số.

> Trong thống kê và lý thuyết xác suất, **phân phối Beta** là một họ của các phân phối xác suất liên tục xác định trên khoảng [0,1] được tham số hóa bởi hai tham số dương, ký hiệu là $\alpha$ và $\beta$, xuất hiện dưới dạng luỹ thừa của biến ngẫu nhiên và kiểm soát hình dạng của phân phối.

>Nếu muốn tìm hiểu nhiều hơn về phân phối Beta thì [đây](http://varianceexplained.org/statistics/beta_distribution_and_baseball/) là bài viết cực kỳ hữu ích.

Logic như sau:


1. Chọn các phân phối tiền nghiệm (prior distribution) cho tham số $\alpha$ và $\beta$.
2. Tính giá trị $\alpha$ và $\beta$: $\alpha = \text{trước + số lượt click (prior + hits)}$, $\beta = \text{trước + số bỏ lỡ (prior + misses)}$. Trong trường hợp này, số lượt click là hits - number of click, số lần hiển thị không có lượt click là misses - number of impression. Phân phối tiền nghiệm (priors) sẽ hữu dụng nếu ta đã có vài thông tin trước về CTR thực tế của các quảng cáo. Nhưng ở đây ta không có, vì vậy ta sẽ dùng 1.0.
3. Ước tính CTR thực tế bằng cách lấy mẫu các giá trị của phân phối Beta cho mỗi biến thể $B(\alpha_i, \beta_i)$ và chọn mẫu với giá trị cao nhất (là CTR ước tính được).
4. Lặp lại bước 2-3.

## Prepare

In [ ]:
np.random.seed(42)

def do_thompson(priors, stat):
  # TODO: Lấy mẫu ctr_0, và ctr_1 dựa trên công thức:
  # 𝛼 = prior + số lượt click
  # 𝛽 = prior + số bỏ lỡ (tức là impressions - clicks)
  ctr_0 = random.betavariate(priors[0]+stat.clicks[0], priors[1] + stat.impressions[0] - stat.clicks[0])
  ctr_1 = random.betavariate(priors[0]+stat.clicks[1], priors[1] + stat.impressions[1] - stat.clicks[1])
  # TODO: Chọn quảng cáo có ctr lấy mẫu cao hơn
  win_index = np.argmax([ctr_0, ctr_1])
  return win_index, (ctr_0, ctr_1)

stat_thompson = Stat()
for i in range(1000):    
    win_index, ctrs = do_thompson(priors, stat_thompson)
    did_click = simulate_customer(ACTUAL_CTR[win_index])
    stat_thompson.record(win_index, did_click, ctrs)

## Benchmark

In [ ]:
## vẽ đồ thị phân phối Beta
x = np.arange (0, 1, 0.01)
y = beta.pdf(x, priors[0]+stat_thompson.clicks[0], priors[1] + stat_thompson.impressions[0] - stat_thompson.clicks[0])
y /= y.max() ## hiệu chỉnh

data1 = go.Scatter(x=x,
                   y=y,
                   name='Beta Distribution (Ad #0)',
                   marker = dict(color=('rgba(10, 108, 94, 1)')),
                   fill='tozeroy',
                   fillcolor = 'rgba(10, 108, 94, .7)')

data2 = go.Scatter(x = [ACTUAL_CTR[0]] * 2,
                   y = [0, 1],
                   name = 'Actual CTR #0 Value',
                   mode='lines',
                   line = dict(
                       color = ('rgb(205, 12, 24)'),
                       width = 2,
                       dash = 'dash'))

y = beta.pdf(x, priors[0]+stat_thompson.clicks[1], priors[1] + stat_thompson.impressions[1] - stat_thompson.clicks[1])
y /= y.max()

data3 = go.Scatter(x=x,
                   y=y,
                   name='Beta Distribution (Ad #1)',
                   marker = dict(color=('rgba(187, 121, 24, 1)')),
                   fill='tozeroy',
                   fillcolor = 'rgba(187, 121, 24, .7)')

data4 = go.Scatter(x = [ACTUAL_CTR[1]] * 2,
                   y = [0, 1],
                   name = 'Actual CTR #1 Value',
                   mode='lines',
                   line = dict(
                       color = ('rgb(205, 12, 24)'),
                       width = 2,
                       dash = 'dash'))

layout = go.Layout(title='Beta Distributions for both Ads',
                   xaxis={'title': 'Possible CTR values'},
                   yaxis={'title': 'Probability Density'})

fig = go.Figure(data=[data1, data2, data3, data4], layout=layout)

iplot(fig, show_link=False)

In [ ]:
algorithm_performance(stat_thompson)

Regret là thấp nhất mà ta từng thấy đến giờ. Mỗi lần tăng regret xảy ra khi quảng cáo #0 được chọn. Trên đồ thị CTR, ta có thể thấy rằng khi bắt đầu, các giá trị có màu xanh (giá trị CTR được lấy mẫu Thompson cho quảng cáo #0) thường cao hơn giá trị c (giá trị CTR được lấy mẫu Thompson cho quảng cáo #1), dẫn đến quảng cáo #0 được chiếu.

Lưu ý về sự khác nhau trong các thay đổi cho mỗi quảng cáo. Thuật toán khai phá liên tục, sau đó khai thác một cách tự nhiên các biến thể quảng cáo với mẫu cao nhất được lấy từ phân phối Beta xấp xỉ. Điều này có thể được biểu diễn ở đồ thị trên cùng của các phân phối. Phân phối Beta cho quảng cáo #1 thì cao hơn nhiều và hẹp hơn nhiều so với quảng cáo #0, đây chính xác là điều ta muốn!

# <a id='comparison'>So sánh và Kết luận</a> 
Đầu tiên, rõ ràng rằng ta muốn chiếu quảng cáo #1 thường xuyên hơn vì CTR thực tế của nó là 0.65. Hãy cùng nhìn xem tỷ lệ số lần quảng cáo đúng được chọn ở mỗi thuật toán.

In [ ]:
random_dict = stat_random.export_stat_dict()
epsilon_dict = stat_epsilon_greedy.export_stat_dict()
ucb_dict = stat_ucb.export_stat_dict()
thompson_dict = stat_thompson.export_stat_dict()


In [ ]:
data1 = go.Bar(x=['Random Selection', 'Epsilon Greedy', 'Thompson Sampling', 'UCB'],
               y=[random_dict['ads_count'][0], 
                  epsilon_dict['ads_count'][0], 
                  thompson_dict['ads_count'][0],
                  ucb_dict['ads_count'][0]],
               name='Ad #0',
               marker=dict(color='rgba(10, 108, 94, .7)'))

data2 = go.Bar(x=['Random Selection', 'Epsilon Greedy', 'Thompson Sampling', 'UCB'],
               y=[random_dict['ads_count'][1], 
                  epsilon_dict['ads_count'][1], 
                  thompson_dict['ads_count'][1],
                  ucb_dict['ads_count'][1]],
               name='Ad #1',
               marker=dict(color='rgba(187, 121, 24, .7)'))

data = [data1, data2]
layout = go.Layout(title='Ratio of appearance of both Ads throughout the trials',
                   xaxis={'title': 'Algorithm'},
                   yaxis={'title': 'Ratio'},
                   barmode='stack')

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Thuật toán trội hơn là Lấy mẫu Thompson và Epsilon-Greedy vì chúng chiếu đúng quảng cáo #1 hầu hết thời gian.

In [ ]:
data1 = go.Scatter(
    x=np.arange (0, n, 1),
    y=random_dict['regret_list'],
    name='Random Selection',
    marker=dict(color='#ffcc66')
)
data2 = go.Scatter(
    x=np.arange (0, n, 1),
    y=epsilon_dict['regret_list'],
    name='e-Greedy',
    marker=dict(color='#0099ff')
)
data3 = go.Scatter(
    x=np.arange (0, n, 1),
    y=thompson_dict['regret_list'],
    name='Thompson Sampling',
    marker=dict(color='#ff3300')
)
data4 = go.Scatter(
    x=np.arange (0, n, 1),
    y=ucb_dict['regret_list'],
    name='UCB',
    marker=dict(color='#33cc33')
)

layout = go.Layout(
    title='Regret by the Algorithm',
    xaxis={'title': 'Trial'},
    yaxis={'title': 'Regret'}
)

data = [data1, data2, data3, data4]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Khi nói đến thuật toán Lấy mẫu Thompson và Epsilon-Greedy đã chọn quảng cáo với CTR cao hơn (#1) trong hầu hết lượt chiết, không ngạc nhiên vì nó cho giá trị regret của chúng là thấp nhất.

In [ ]:
data = go.Bar(
    x=[ucb_dict['reward'], thompson_dict['reward'], epsilon_dict['reward'], random_dict['reward']],
    y=['UCB', 'Thompson Sampling', 'e-Greedy','Random Selection'],
    orientation = 'h',
    marker=dict(color=['#33cc33', '#ff3300', '#0099ff', '#ffcc66']),
    opacity=0.7
)

text = go.Scatter(
    x=[ucb_dict['reward'], thompson_dict['reward'], epsilon_dict['reward'], random_dict['reward']],
    y=['UCB', 'Thompson Sampling', 'e-Greedy', 'Random Selection'],
    mode='text',
    text=[ucb_dict['reward'], thompson_dict['reward'], epsilon_dict['reward'], random_dict['reward']],
    textposition='middle left',
    line = dict(
        color = ('rgba(255,141,41,0.6)'),
        width = 1
    ),
    textfont=dict(
        family='sans serif',
        size=16,
        color='#000000'
    )
)

data = [data,text]

layout = go.Layout(
    title='Total Reward by Algorithms',
    xaxis={'title': 'Total Reward (Clicks)'},
    margin={'l':200},
    showlegend=False
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Có thể trường hợp mà tổng reward cho thuật toán với giá trị regret thấp nhất sẽ không phải là cao nhất. Điều này là bởi thực tế rằng: dù cho thuật toán chọn ra được đúng quảng cáo thì nó cũng không đảm bảo rằng khách hàng sẽ click vào xem.